# Fine-tuning 5th. Workflow (2nd + 캐글 noraml dataset + ai-hub 12000장 damaged 적용)
1. 하이브리드 라벨링 전략 (Hybrid Labeling Strategy)
    - 1단계 (우선순위): Auto-Labeling (YOLOv8x)
        - 일단 Pre-trained 모델로 "차량 전체 형상"을 찾기
        - 성공 시: Normal 데이터와 기준이 같아지므로 베스트
    - 2단계 (Fallback): JSON 라벨 활용
        - 만약 모델이 너무 확대된(Zoom-in) 이미지라 차량을 못 찾으면(Empty), 그때 JSON의 파손 부위 좌표를 가져옴
    - 이유: "못 찾았다고 빈 파일(Background)"로 두면 안 됩니다. Damaged 폴더에 있으므로 무조건. 부분만이라도 '차량'이라고 가르쳐야

2. 데이터 분할 및 격리
 - 모델의 암기(Memorizing)를 방지하고 객관적인 성능 검증을 위한 데이터 분리.
 - 비율 (Ratio): 전체 데이터를 7 : 2 : 1 비율로 랜덤 분할.
    - Train (70%): 모델 가중치 업데이트용 (학습).
    - Val (20%): 학습 중 성능 모니터링 및 조기 종료(Early Stopping) 결정용.
    - Test (10%): 완전 격리(Isolation). 학습 과정에 절대 관여하지 않으며, 최종 성능 평가에만 사용.

3. 학습 환경 설정 (Configuration)
 - YOLO 모델이 데이터셋의 위치와 클래스 정보를 알 수 있도록 설정.
 - yolov8_tuning_251207.yaml 생성: Train/Val 경로 및 클래스명(Vehicle) 정의 (Test 경로 제외)
 - Hyperparameter:
    - Freeze: Pre-trained 가중치 앞단을 고정하여 기존의 일반적인 특징 추출 능력 유지.
    - Epochs: 15~20회 (과적합 방지를 위해 적절한 횟수 설정).
    - Learning Rate: 미세 조정을 위해 낮은 학습률 적용.

4. 학습 수행 (Fine-tuning Execution)
 - 준비된 데이터(Train/Val)를 통해 모델이 새로운 도메인(파손, 줌인 이미지) 적응.
 - process
    - Pre-trained yolov8x.pt 로드.
    - 손실 함수(Loss Function)가 최소화되는 방향으로 가중치 업데이트.
    - 결과물: 우리 데이터에 최적화된 best.pt 모델 생성

 5. 최종 성능 평가 (Final Evaluation)
 - 학습에 사용되지 않은 Test Set을 사용하여 실전 성능 검증.
 - 검증 방법
    - 격리해둔 Test Set(10%)에 대해 추론 수행.
    - 기존 Pre-trained 모델 대비 False Negative(미검출) 감소량 확인.
    - 주요 지표: Accuracy(정확도), Recall(재현율), Confusion Matrix(혼동 행렬) 비교 분석.
 6. (추가) damaged 파일 추가
  - (기존) 기존 ai-hub 샘플 1200장 damaged 파일
  - (개선) ai-hub 원본 50만장에서 파손 종류(4)에 따라 동일한 비율로 12000장 적용

# 라이브러리 설치

In [ ]:
# [Cell 1] 라이브러리 설치
# YOLOv8 및 데이터 처리에 필요한 라이브러리 설치
!pip install ultralytics tqdm

import os
import shutil
import json
import random
import yaml
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from ultralytics import YOLO
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from google.colab import drive
import glob
import cv2

# 구글 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

print("✅ 환경 설정 완료")

# 데이터셋 구축 및 분할 1차
 - 캐글 데이터셋 추가 + ai-hub 12000장 damaged 파일
 - confidence threshold 0.5

In [ ]:
# 1. 입력 데이터 경로 (Source)
# (1) AI-Hub (Damaged)
AI_HUB_IMG_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/AI_HUB_DAMAGE_DATASET/images"
AI_HUB_LBL_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/AI_HUB_DAMAGE_DATASET/labels"

# (2) Normal & Background (Source Base)
NORMAL_ROOT_BASE = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/RAW"

# 2. 결과 데이터셋 저장 경로 (Destination)
DEST_DIR = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"

## 반복 실행 금지(x)

In [ ]:
# [Cell 1] (수정본) 환경 설정 및 하이브리드 데이터셋 구축
# 수정 사항: background 폴더 추가 및 'empty' 전략 적용

from google.colab import drive
import os
import glob
import shutil
import random
import yaml
from tqdm import tqdm
from ultralytics import YOLO

# 1. 구글 드라이브 마운트 (이미 되어 있으면 건너뜀)
# drive.mount('/content/drive')

# ==============================================================================
# ⚙️ [설정] 경로 및 파라미터 정의
# ==============================================================================

# 1. 입력 데이터 경로 (Source)
# (1) AI-Hub (Damaged)
AI_HUB_IMG_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/AI_HUB_DAMAGE_DATASET/images"
AI_HUB_LBL_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/AI_HUB_DAMAGE_DATASET/labels"

# (2) Normal & Background (Source Base)
NORMAL_ROOT_BASE = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/RAW"

# 2. 결과 데이터셋 저장 경로 (Destination)
DEST_DIR = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"

# 3. 모델 설정 (Auto-Labeling용)
AUTO_LABEL_MODEL = 'yolov8x.pt'

# 4. 데이터 분할 비율
SPLIT_RATIO = {'train': 0.7, 'val': 0.2, 'test': 0.1}

# ==============================================================================

def setup_directories():
    """저장소 폴더 구조 초기화"""
    if os.path.exists(DEST_DIR):
        print(f"⚠️ 기존 데이터셋 폴더가 존재합니다. 삭제 후 재생성합니다: {DEST_DIR}")
        shutil.rmtree(DEST_DIR)

    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(DEST_DIR, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(DEST_DIR, 'labels', split), exist_ok=True)
    print(f"✅ 폴더 구조 생성 완료")

def get_target_image_paths():
    """대상 이미지 경로 수집 (Background 포함)"""
    print("🔍 이미지 경로 수집 중...")

    # 1. AI-Hub Images (Damaged)
    damaged_imgs = glob.glob(os.path.join(AI_HUB_IMG_ROOT, "**", "*.jpg"), recursive=True)
    damaged_imgs += glob.glob(os.path.join(AI_HUB_IMG_ROOT, "**", "*.png"), recursive=True)

    # 2. Normal & Background Images (지정된 3개 폴더)
    # [수정] background 폴더 추가
    target_folders = ['normal', 'normal(kaggle_dataset)', 'background']
    normal_bg_imgs = []

    for folder_name in target_folders:
        folder_path = os.path.join(NORMAL_ROOT_BASE, folder_name)
        if os.path.exists(folder_path):
            imgs = glob.glob(os.path.join(folder_path, "**", "*.jpg"), recursive=True)
            imgs += glob.glob(os.path.join(folder_path, "**", "*.png"), recursive=True)
            print(f"   👉 Folder '{folder_name}': {len(imgs)}장")
            normal_bg_imgs += imgs
        else:
            print(f"   ⚠️ 경고: 폴더를 찾을 수 없습니다: {folder_path}")

    print(f"   -------------------------------------------")
    print(f"   ✅ AI-Hub (Damaged): {len(damaged_imgs)}장")
    print(f"   ✅ Normal/Background: {len(normal_bg_imgs)}장")
    print(f"   -------------------------------------------")

    all_imgs = damaged_imgs + normal_bg_imgs
    random.shuffle(all_imgs)
    return all_imgs

def get_fallback_label_from_txt(img_path):
    """[Fallback] AI-Hub 데이터인 경우, 기존 TXT 라벨을 찾아 '0' 클래스로 변환 반환"""
    file_name = os.path.basename(img_path)
    txt_name = os.path.splitext(file_name)[0] + ".txt"

    # 경로 매칭 및 검색
    try:
        rel_path = os.path.relpath(os.path.dirname(img_path), AI_HUB_IMG_ROOT)
        target_txt_path = os.path.join(AI_HUB_LBL_ROOT, rel_path, txt_name)
    except:
        target_txt_path = ""

    if not os.path.exists(target_txt_path):
        found = glob.glob(os.path.join(AI_HUB_LBL_ROOT, "**", txt_name), recursive=True)
        if found: target_txt_path = found[0]
        else: return None

    try:
        with open(target_txt_path, 'r') as f:
            lines = f.readlines()
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                # 클래스 ID -> '0' (Vehicle)
                new_line = f"0 {parts[1]} {parts[2]} {parts[3]} {parts[4]}"
                new_lines.append(new_line)
        return new_lines if new_lines else None
    except:
        return None

def run_hybrid_labeling(img_paths):
    print(f"🚀 Hybrid Labeling 시작 (총 {len(img_paths)}장)...")
    print("   (Strategies: damaged=hybrid, normal=auto, background=empty)")

    model = YOLO(AUTO_LABEL_MODEL)
    stats = {'auto': 0, 'fallback': 0, 'bg': 0, 'skip': 0,
             'train': 0, 'val': 0, 'test': 0}

    for img_path in tqdm(img_paths):
        label_data = []

        # 경로 분석으로 소스 판별
        # 주의: 폴더명 매칭 시 os.sep(/)을 포함하거나 split해서 정확히 비교
        path_parts = img_path.split(os.sep)

        # [수정] 전략 분기
        if 'background' in path_parts:
            # 전략: "empty" (강제 배경 처리)
            # YOLO 돌리지 않고 바로 빈 라벨 생성
            label_data = []
            stats['bg'] += 1

        elif ('normal' in path_parts) or ('normal(kaggle_dataset)' in path_parts):
            # 전략: "auto_label" (Normal)
            # YOLO로 차 찾기 -> 없으면 Background(empty)
            results = model.predict(source=img_path, conf=0.5, classes=[2, 5, 7], verbose=False)
            if len(results[0].boxes) > 0:
                for box in results[0].boxes:
                    x, y, w, h = box.xywhn[0].tolist()
                    label_data.append(f"0 {x:.6f} {y:.6f} {w:.6f} {h:.6f}")
                stats['auto'] += 1
            else:
                label_data = [] # 못 찾으면 배경으로 활용
                stats['bg'] += 1

        else:
            # 전략: "hybrid" (AI-Hub Damaged)
            # 1. Auto-Labeling 시도
            results = model.predict(source=img_path, conf=0.5, classes=[2, 5, 7], verbose=False)
            if len(results[0].boxes) > 0:
                for box in results[0].boxes:
                    x, y, w, h = box.xywhn[0].tolist()
                    label_data.append(f"0 {x:.6f} {y:.6f} {w:.6f} {h:.6f}")
                stats['auto'] += 1
            else:
                # 2. Fallback (기존 TXT)
                fallback_lines = get_fallback_label_from_txt(img_path)
                if fallback_lines:
                    label_data = fallback_lines
                    stats['fallback'] += 1
                else:
                    stats['skip'] += 1
                    continue

        # --- 분할 및 저장 ---
        rand = random.random()
        if rand < SPLIT_RATIO['train']: split = 'train'
        elif rand < SPLIT_RATIO['train'] + SPLIT_RATIO['val']: split = 'val'
        else: split = 'test'

        file_name = os.path.basename(img_path)
        shutil.copy2(img_path, os.path.join(DEST_DIR, 'images', split, file_name))

        txt_name = os.path.splitext(file_name)[0] + ".txt"
        with open(os.path.join(DEST_DIR, 'labels', split, txt_name), 'w') as f:
            if label_data:
                f.write("\n".join(label_data))
        stats[split] += 1

    print("\n✅ 데이터셋 구축 완료!")
    print(f"   [Labeling] Auto: {stats['auto']} | Fallback: {stats['fallback']} | BG(Empty): {stats['bg']} | Skip: {stats['skip']}")
    print(f"   [Split] Train: {stats['train']} | Val: {stats['val']} | Test: {stats['test']}")

def create_yaml():
    yaml_txt = f"""
    path: {DEST_DIR}
    train: images/train
    val: images/val
    test: images/test
    names:
      0: Vehicle
    """
    path = os.path.join(DEST_DIR, 'data.yaml')
    with open(path, 'w') as f:
        f.write(yaml_txt)
    print(f"✅ YAML 파일 생성 완료: {path}")
    return path

# --- 실행 ---
setup_directories()
target_images = get_target_image_paths()
run_hybrid_labeling(target_images)
YAML_PATH_GENERATED = create_yaml()

In [ ]:
# [Cell 2] 데이터셋 현황 검증 (Verification)
import os
import glob

def print_dataset_stats(base_dir):
    print("="*60)
    print(f"📊 [데이터셋 검증 리포트] : {base_dir}")
    print("="*60)

    splits = ['train', 'val', 'test']
    total_imgs = 0
    total_lbls = 0
    total_bg = 0

    for split in splits:
        img_dir = os.path.join(base_dir, 'images', split)
        lbl_dir = os.path.join(base_dir, 'labels', split)

        # 파일 개수 확인
        n_imgs = len(glob.glob(os.path.join(img_dir, "*.*")))
        txt_files = glob.glob(os.path.join(lbl_dir, "*.txt"))
        n_lbls = len(txt_files)

        # 빈 라벨 파일(Background) 확인
        n_empty_lbls = 0
        for txt in txt_files:
            if os.path.getsize(txt) == 0:
                n_empty_lbls += 1

        print(f"📂 [{split.upper()}]")
        print(f"   - Images : {n_imgs}장")
        print(f"   - Labels : {n_lbls}개")
        print(f"     ㄴ 유효 라벨(Vehicle) : {n_lbls - n_empty_lbls}개")
        print(f"     ㄴ 빈 라벨(Background): {n_empty_lbls}개 (오탐지 방지용)")

        total_imgs += n_imgs
        total_lbls += n_lbls
        total_bg += n_empty_lbls
        print("-" * 40)

    print(f"📈 [TOTAL SUMMARY]")
    print(f"   - 총 이미지 수 : {total_imgs}장")
    print(f"   - Background 비율: {(total_bg/total_imgs)*100:.1f}% ({total_bg}장)")
    print(f"   - 저장 위치: {base_dir}")
    print("="*60)

# 검증 실행
print_dataset_stats(DEST_DIR)

# Fine-tuning 5차
 - EPOCHS = 50       
 - BATCH_SIZE = 16
 - IMG_SIZE = 640
 - lr0=1e-5,
 - freeze=10,   
 - patience=15,
 - + 캐글 normal 데이터셋

In [ ]:
### train시 1시간 정도 소모 됩니다. colab환경에서 아래를 browser의 console 에서 붙여 넣기가 필요할 수 있습니다.
### shift+cntr+i 로 browser console 열기
# https://github.com/chulminkw/DLCV/blob/master/data/util/colab_autoclick.js
'''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''

In [ ]:
AI_HUB_IMG_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/AI_HUB_DAMAGE_DATASET/images"
AI_HUB_LBL_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/AI_HUB_DAMAGE_DATASET/labels"

# (2) Normal & Background (Source Base)
NORMAL_ROOT_BASE = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/RAW"

# 2. 결과 데이터셋 저장 경로 (Destination)
DEST_DIR = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"

In [ ]:
# [Cell 3] Fine-tuning 시작 (수정됨: Epoch 50회 + 구글 드라이브 저장)
import os
import shutil
import json
import random
from ultralytics import YOLO
from tqdm import tqdm

# =========================================================
# [설정] 경로
# =========================================================
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"
PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION"

# =========================================================
# [설정] 파라미터
# =========================================================
YAML_PATH = os.path.join(DATASET_ROOT, 'data.yaml')
MODEL_NAME = 'yolov8x.pt'
EPOCHS = 100               # [수정 1] Epoch를 15 -> 50으로 변경
BATCH_SIZE = 16
IMG_SIZE = 640

def run_training():
    # [수정 2] 구글 드라이브 저장 경로 설정
    SAVE_DIR = os.path.join(PROJECT_ROOT, "FINE_TUNING_MODEL")

    print(f"🔥 학습 시작: {MODEL_NAME} -> Epochs: {EPOCHS}")
    print(f"   💾 결과 저장 경로: {SAVE_DIR}")

    # 모델 로드
    model = YOLO(MODEL_NAME)

    # 학습 실행
    model.train(
        data=YAML_PATH,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,

        # [수정 3] project 파라미터 추가 (여기에 지정한 폴더 안에 결과가 저장됨)
        project=SAVE_DIR,
        name='yolov8x_fine_tuning_5th', # 최종 경로: .../train_results/yolov8x_finetuned_vehicle
        exist_ok=True,
        optimizer='AdamW',
        freeze=10,        # 기존 설정 유지
        patience=10,
        lr0=1e-4,             # Fine-tuning이므로 학습률을 조금 낮게 설정
        close_mosaic=10,
        verbose=True
    )
    print(f"🎉 학습 완료!")
    print(f"   결과가 드라이브에 저장되었습니다: {os.path.join(SAVE_DIR, 'yolov8x_finetuned_vehicle')}")

# 실행
run_training()

## test set 성능 평가(0.25)

In [ ]:
# [Cell 4] Fine-tuned Model Test Set 평가 (경로 수정됨)
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm

# =========================================================
# [설정] 경로 및 파라미터 (User 환경에 맞게 자동 수정됨)
# =========================================================
# 1. 학습된 모델 경로
#    (방금 학습한 결과가 저장된 구글 드라이브 경로)
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"

MODEL_PATH = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/FINE_TUNING_MODEL/yolov8x_fine_tuning_5th/weights/best.pt"

# 2. 테스트 데이터셋 경로 (Hybrid_Split_721 폴더 내의 test 폴더)
TEST_IMG_DIR = os.path.join(DATASET_ROOT, "images/test")
TEST_LABEL_DIR = os.path.join(DATASET_ROOT, "labels/test")

# 3. 결과 저장 경로
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

CONF_THRESHOLD = 0.25  # Fine-tuning 했으므로 표준 임계값 사용

def evaluate_finetuned_model():
    if not os.path.exists(MODEL_PATH):
        print(f"❌ 모델 파일이 없습니다: {MODEL_PATH}")
        print("   -> 학습이 정상적으로 완료되었는지 확인해주세요.")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {MODEL_PATH}")
    print(f"   - 데이터: {TEST_IMG_DIR}")

    model = YOLO(MODEL_PATH)

    # 평가 대상 파일 리스트업
    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    if len(image_files) == 0:
        print(f"❌ 테스트 이미지가 없습니다. 경로를 확인해주세요: {TEST_IMG_DIR}")
        return

    results_list = []
    speed_stats = []

    print(f"   - 총 평가 대상: {len(image_files)}장")

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        label_path = os.path.join(TEST_LABEL_DIR, os.path.splitext(file)[0] + ".txt")

        # -----------------------------------------------------
        # 1. 정답(True Label) 확인 로직
        # -----------------------------------------------------
        # 라벨 파일이 있고, 내용이 비어있지 않으면 Vehicle(1)
        true_label = 0
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                content = f.read().strip()
                if len(content) > 0:
                    true_label = 1

        # -----------------------------------------------------
        # 2. 모델 추론 (Prediction)
        # -----------------------------------------------------
        results = model(img_path, conf=CONF_THRESHOLD, verbose=False)

        # 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # 예측 라벨 결정 (박스가 하나라도 있으면 Vehicle)
        pred_label = 0
        for r in results:
            if len(r.boxes) > 0:
                pred_label = 1
                break

        # 결과 저장
        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label),
            "full_path": img_path
        })

    # 데이터프레임 변환
    df = pd.DataFrame(results_list)

    # =========================================================
    # [분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # A. 정확도 평가
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"✅ 1. 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # B. 속도 평가
    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time

        print(f"\n⚡ 2. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # C. 상세 분류 리포트
    print("\n📝 3. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Non-Vehicle', 'Vehicle']))

    # D. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Non-Vehicle', 'Pred: Vehicle'],
                yticklabels=['True: Non-Vehicle', 'True: Vehicle'])
    plt.title(f'Confusion Matrix\n(Fine-tuned Result)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # =========================================================
    # E. 결과 저장
    # =========================================================
    save_filename = f"inference_finetuned_5th_test_results.csv"
    save_path = os.path.join(RESULT_ROOT, save_filename)

    df.to_csv(save_path, index=False, encoding='utf-8-sig')

    print(f"\n💾 전체 실행 결과 저장 완료!")
    print(f"   -> 파일 경로: {save_path}")
    print(f"   -> 총 데이터 수: {len(df)}건")

    failed_count = len(df[df['is_correct'] == False])
    print(f"   -> (성공: {len(df)-failed_count}건, 실패: {failed_count}건)")

# 실행
evaluate_finetuned_model()

## 오탐 대상 시각화

In [ ]:
# [Cell] 오탐(Failure Case) 시각화 분석
import cv2
import matplotlib.pyplot as plt

DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"
MODEL_PATH = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/FINE_TUNING_MODEL/yolov8x_fine_tuning_5th/weights/best.pt"

# 2. 테스트 데이터셋 경로 (Hybrid_Split_721 폴더 내의 test 폴더)
TEST_IMG_DIR = os.path.join(DATASET_ROOT, "images/test")
TEST_LABEL_DIR = os.path.join(DATASET_ROOT, "labels/test")

# 3. 결과 저장 경로
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/RESULT"
CONF_THRESHOLD = 0.25  # Fine-tuning 했으므로 표준 임계값 사용


def visualize_failures(num_samples=5):
    # 1. 저장된 결과 CSV 로드
    csv_path = os.path.join(RESULT_ROOT, "inference_finetuned_5th_test_results.csv")

    if not os.path.exists(csv_path):
        print(f"❌ 결과 파일이 없습니다. 먼저 평가 코드를 실행하세요: {csv_path}")
        return

    df = pd.read_csv(csv_path)

    # 2. 실패 케이스 필터링 (is_correct == False)
    failures = df[df['is_correct'] == False]

    if len(failures) == 0:
        print("🎉 오탐(실패) 케이스가 없습니다! 완벽합니다.")
        return

    print(f"🔍 총 {len(failures)}건의 오탐 중 상위 {min(len(failures), num_samples)}건을 시각화합니다...")

    # 모델 로드 (박스 시각화를 위해 필요)
    model = YOLO(MODEL_PATH)

    # 샘플링
    samples = failures.head(num_samples)

    # 시각화 설정
    plt.figure(figsize=(12, 8 * len(samples)))

    for i, (_, row) in enumerate(samples.iterrows()):
        img_path = row['full_path']
        filename = row['filename']
        true_label = row['true_label'] # 1: Vehicle, 0: Non-Vehicle
        pred_label = row['pred_label']

        # 이미지 로드
        if not os.path.exists(img_path):
            print(f"⚠️ 이미지 파일 없음: {img_path}")
            continue

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # -------------------------------------------------
        # A. Ground Truth 그리기 (Green Box)
        # -------------------------------------------------
        label_file = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(TEST_LABEL_DIR, label_file)

        gt_desc = "Non-Vehicle"
        if true_label == 1:
            gt_desc = "Vehicle"
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    for line in lines:
                        # YOLO format: class x_center y_center w h
                        parts = list(map(float, line.strip().split()))
                        if len(parts) >= 5:
                            _, bx, by, bw, bh = parts
                            # 좌표 변환 (Normalized -> Pixel)
                            x1 = int((bx - bw/2) * w)
                            y1 = int((by - bh/2) * h)
                            x2 = int((bx + bw/2) * w)
                            y2 = int((by + bh/2) * h)

                            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 5) # 초록색 박스

        # -------------------------------------------------
        # B. Prediction 그리기 (Red Box)
        # -------------------------------------------------
        # 시각화를 위해 해당 이미지만 다시 추론
        results = model(img_path, conf=CONF_THRESHOLD, verbose=False)

        pred_desc = "Non-Vehicle"
        has_pred_box = False

        for r in results:
            for box in r.boxes:
                has_pred_box = True
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])

                # 예측 박스 그리기 (Red)
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
                # 신뢰도 표시
                label_text = f"{conf:.2f}"
                t_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
                cv2.rectangle(img, (x1, y1-30), (x1+t_size[0], y1), (255, 0, 0), -1)
                cv2.putText(img, label_text, (x1, y1-5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        if has_pred_box:
            pred_desc = "Vehicle"

        # -------------------------------------------------
        # C. 출력
        # -------------------------------------------------
        plt.subplot(len(samples), 1, i+1)
        plt.imshow(img)
        plt.axis('off')

        # 제목 설정 (실패 원인 명시)
        fail_type = "False Negative (미검출)" if true_label == 1 else "False Positive (오검출)"
        title_text = f"[{i+1}] {filename}\nType: {fail_type}\nGT(Green): {gt_desc} vs Pred(Red): {pred_desc}"

        plt.title(title_text, color='red', fontsize=15, fontweight='bold', pad=20)

    plt.tight_layout()
    plt.show()

# 실행
visualize_failures(130)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import os
import random
from ultralytics import YOLO

# =========================================================
# [설정] 경로
# =========================================================
# 1. 학습된 최적 모델 경로 (확인 필요)
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"
MODEL_PATH = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/FINE_TUNING_MODEL/yolov8x_fine_tuning_5th/weights/best.pt"

# 2. 테스트 데이터셋 경로 (Hybrid_Split_721 폴더 내의 test 폴더)
TEST_IMG_DIR = os.path.join(DATASET_ROOT, "images/test")
TEST_LABEL_DIR = os.path.join(DATASET_ROOT, "labels/test")

def visualize_comparison(num_samples=5):
    if not os.path.exists(MODEL_PATH):
        print("❌ 학습된 모델 파일이 없습니다.")
        return

    model = YOLO(MODEL_PATH)
    files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png'))]
    random.shuffle(files)

    samples = files[:num_samples]

    plt.figure(figsize=(15, 5 * num_samples))

    for i, file in enumerate(samples):
        img_path = os.path.join(TEST_IMG_DIR, file)
        label_path = os.path.join(TEST_LABEL_DIR, os.path.splitext(file)[0] + ".txt")

        # 이미지 로드
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # -------------------------------------------------
        # 1. Ground Truth (초록색) 그리기
        # -------------------------------------------------
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls, x, y, bw, bh = map(float, line.strip().split())
                    # YOLO xywhn -> pixel xymn (좌상단)
                    x1 = int((x - bw / 2) * w)
                    y1 = int((y - bh / 2) * h)
                    x2 = int((x + bw / 2) * w)
                    y2 = int((y + bh / 2) * h)

                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3) # Green
                    cv2.putText(img, "GT", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # -------------------------------------------------
        # 2. Prediction (빨간색) 그리기
        # -------------------------------------------------
        results = model(img_path, conf=0.1, verbose=False) # Threshold 낮게 설정
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])

                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3) # Red
                cv2.putText(img, f"Pred {conf:.2f}", (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # 출력
        plt.subplot(num_samples, 1, i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Sample: {file} (Green: GT, Red: Prediction)", fontsize=14)

    plt.tight_layout()
    plt.show()


In [ ]:
# 실행
visualize_comparison(1)

## test set 성능 평가(0.1)

In [ ]:
# [Cell 4] Fine-tuned Model Test Set 평가 (경로 수정됨)
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm

# =========================================================
# [설정] 경로 및 파라미터 (User 환경에 맞게 자동 수정됨)
# =========================================================
# 1. 학습된 모델 경로
#    (방금 학습한 결과가 저장된 구글 드라이브 경로)
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"

MODEL_PATH = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/FINE_TUNING_MODEL/yolov8x_fine_tuning_5th/weights/best.pt"

# 2. 테스트 데이터셋 경로 (Hybrid_Split_721 폴더 내의 test 폴더)
TEST_IMG_DIR = os.path.join(DATASET_ROOT, "images/test")
TEST_LABEL_DIR = os.path.join(DATASET_ROOT, "labels/test")

# 3. 결과 저장 경로
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/RESULT"
os.makedirs(RESULT_ROOT, exist_ok=True)

CONF_THRESHOLD = 0.1  # Fine-tuning 했으므로 표준 임계값 사용

def evaluate_finetuned_model():
    if not os.path.exists(MODEL_PATH):
        print(f"❌ 모델 파일이 없습니다: {MODEL_PATH}")
        print("   -> 학습이 정상적으로 완료되었는지 확인해주세요.")
        return

    print(f"🚀 평가 시작...")
    print(f"   - 모델: {MODEL_PATH}")
    print(f"   - 데이터: {TEST_IMG_DIR}")

    model = YOLO(MODEL_PATH)

    # 평가 대상 파일 리스트업
    image_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    if len(image_files) == 0:
        print(f"❌ 테스트 이미지가 없습니다. 경로를 확인해주세요: {TEST_IMG_DIR}")
        return

    results_list = []
    speed_stats = []

    print(f"   - 총 평가 대상: {len(image_files)}장")

    for file in tqdm(image_files, desc="Inference"):
        img_path = os.path.join(TEST_IMG_DIR, file)
        label_path = os.path.join(TEST_LABEL_DIR, os.path.splitext(file)[0] + ".txt")

        # -----------------------------------------------------
        # 1. 정답(True Label) 확인 로직
        # -----------------------------------------------------
        # 라벨 파일이 있고, 내용이 비어있지 않으면 Vehicle(1)
        true_label = 0
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                content = f.read().strip()
                if len(content) > 0:
                    true_label = 1

        # -----------------------------------------------------
        # 2. 모델 추론 (Prediction)
        # -----------------------------------------------------
        results = model(img_path, conf=CONF_THRESHOLD, verbose=False)

        # 속도 측정
        speed_info = results[0].speed
        total_time_ms = speed_info['preprocess'] + speed_info['inference'] + speed_info['postprocess']
        speed_stats.append(total_time_ms)

        # 예측 라벨 결정 (박스가 하나라도 있으면 Vehicle)
        pred_label = 0
        for r in results:
            if len(r.boxes) > 0:
                pred_label = 1
                break

        # 결과 저장
        results_list.append({
            "filename": file,
            "true_label": true_label,
            "pred_label": pred_label,
            "is_correct": (true_label == pred_label),
            "full_path": img_path
        })

    # 데이터프레임 변환
    df = pd.DataFrame(results_list)

    # =========================================================
    # [분석 결과 리포트]
    # =========================================================
    print("\n" + "="*50)
    print("📊 [최종 분석 결과 - Fine-tuned Model]")
    print("="*50)

    # A. 정확도 평가
    acc = accuracy_score(df['true_label'], df['pred_label'])
    print(f"✅ 1. 정확도 (Accuracy): {acc:.4f} ({acc*100:.2f}%)")

    # B. 속도 평가
    if speed_stats:
        avg_time = np.mean(speed_stats)
        min_time = np.min(speed_stats)
        max_time = np.max(speed_stats)
        fps = 1000 / avg_time

        print(f"\n⚡ 2. 추론 속도 (Inference Speed):")
        print(f"   - 평균 소요 시간 : {avg_time:.2f} ms/장")
        print(f"   - 최소 소요 시간 : {min_time:.2f} ms")
        print(f"   - 최대 소요 시간 : {max_time:.2f} ms")
        print(f"   - 처리량 (FPS)   : {fps:.2f} FPS")

    # C. 상세 분류 리포트
    print("\n📝 3. 상세 분류 리포트:")
    print(classification_report(df['true_label'], df['pred_label'], target_names=['Non-Vehicle', 'Vehicle']))

    # D. 혼동 행렬 시각화
    plt.figure(figsize=(6, 5))
    cm = confusion_matrix(df['true_label'], df['pred_label'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Pred: Non-Vehicle', 'Pred: Vehicle'],
                yticklabels=['True: Non-Vehicle', 'True: Vehicle'])
    plt.title(f'Confusion Matrix\n(Fine-tuned Result)')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

    # =========================================================
    # E. 결과 저장
    # =========================================================
    save_filename = f"inference_finetuned_5th_test_results_conf01.csv"
    save_path = os.path.join(RESULT_ROOT, save_filename)

    df.to_csv(save_path, index=False, encoding='utf-8-sig')

    print(f"\n💾 전체 실행 결과 저장 완료!")
    print(f"   -> 파일 경로: {save_path}")
    print(f"   -> 총 데이터 수: {len(df)}건")

    failed_count = len(df[df['is_correct'] == False])
    print(f"   -> (성공: {len(df)-failed_count}건, 실패: {failed_count}건)")

# 실행
evaluate_finetuned_model()

## 오탐 대상 시각화

In [ ]:
# [Cell] 오탐(Failure Case) 시각화 분석
import cv2
import matplotlib.pyplot as plt

DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"
MODEL_PATH = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/FINE_TUNING_MODEL/yolov8x_fine_tuning_5th/weights/best.pt"

# 2. 테스트 데이터셋 경로 (Hybrid_Split_721 폴더 내의 test 폴더)
TEST_IMG_DIR = os.path.join(DATASET_ROOT, "images/test")
TEST_LABEL_DIR = os.path.join(DATASET_ROOT, "labels/test")

# 3. 결과 저장 경로
RESULT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/RESULT"
CONF_THRESHOLD = 0.25  # Fine-tuning 했으므로 표준 임계값 사용


def visualize_failures(num_samples=5):
    # 1. 저장된 결과 CSV 로드
    csv_path = os.path.join(RESULT_ROOT, "inference_finetuned_5th_test_results_conf01.csv")

    if not os.path.exists(csv_path):
        print(f"❌ 결과 파일이 없습니다. 먼저 평가 코드를 실행하세요: {csv_path}")
        return

    df = pd.read_csv(csv_path)

    # 2. 실패 케이스 필터링 (is_correct == False)
    failures = df[df['is_correct'] == False]

    if len(failures) == 0:
        print("🎉 오탐(실패) 케이스가 없습니다! 완벽합니다.")
        return

    print(f"🔍 총 {len(failures)}건의 오탐 중 상위 {min(len(failures), num_samples)}건을 시각화합니다...")

    # 모델 로드 (박스 시각화를 위해 필요)
    model = YOLO(MODEL_PATH)

    # 샘플링
    samples = failures.head(num_samples)

    # 시각화 설정
    plt.figure(figsize=(12, 8 * len(samples)))

    for i, (_, row) in enumerate(samples.iterrows()):
        img_path = row['full_path']
        filename = row['filename']
        true_label = row['true_label'] # 1: Vehicle, 0: Non-Vehicle
        pred_label = row['pred_label']

        # 이미지 로드
        if not os.path.exists(img_path):
            print(f"⚠️ 이미지 파일 없음: {img_path}")
            continue

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # -------------------------------------------------
        # A. Ground Truth 그리기 (Green Box)
        # -------------------------------------------------
        label_file = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(TEST_LABEL_DIR, label_file)

        gt_desc = "Non-Vehicle"
        if true_label == 1:
            gt_desc = "Vehicle"
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    for line in lines:
                        # YOLO format: class x_center y_center w h
                        parts = list(map(float, line.strip().split()))
                        if len(parts) >= 5:
                            _, bx, by, bw, bh = parts
                            # 좌표 변환 (Normalized -> Pixel)
                            x1 = int((bx - bw/2) * w)
                            y1 = int((by - bh/2) * h)
                            x2 = int((bx + bw/2) * w)
                            y2 = int((by + bh/2) * h)

                            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 5) # 초록색 박스

        # -------------------------------------------------
        # B. Prediction 그리기 (Red Box)
        # -------------------------------------------------
        # 시각화를 위해 해당 이미지만 다시 추론
        results = model(img_path, conf=CONF_THRESHOLD, verbose=False)

        pred_desc = "Non-Vehicle"
        has_pred_box = False

        for r in results:
            for box in r.boxes:
                has_pred_box = True
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])

                # 예측 박스 그리기 (Red)
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
                # 신뢰도 표시
                label_text = f"{conf:.2f}"
                t_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
                cv2.rectangle(img, (x1, y1-30), (x1+t_size[0], y1), (255, 0, 0), -1)
                cv2.putText(img, label_text, (x1, y1-5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        if has_pred_box:
            pred_desc = "Vehicle"

        # -------------------------------------------------
        # C. 출력
        # -------------------------------------------------
        plt.subplot(len(samples), 1, i+1)
        plt.imshow(img)
        plt.axis('off')

        # 제목 설정 (실패 원인 명시)
        fail_type = "False Negative (미검출)" if true_label == 1 else "False Positive (오검출)"
        title_text = f"[{i+1}] {filename}\nType: {fail_type}\nGT(Green): {gt_desc} vs Pred(Red): {pred_desc}"

        plt.title(title_text, color='red', fontsize=15, fontweight='bold', pad=20)

    plt.tight_layout()
    plt.show()

# 실행
visualize_failures(130)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import os
import random
from ultralytics import YOLO

# =========================================================
# [설정] 경로
# =========================================================
# 1. 학습된 최적 모델 경로 (확인 필요)
DATASET_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/DATA/CAR_DETECTION_AIHUB_KAGGLE"
MODEL_PATH = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/FINE_TUNING_MODEL/yolov8x_fine_tuning_5th/weights/best.pt"

# 2. 테스트 데이터셋 경로 (Hybrid_Split_721 폴더 내의 test 폴더)
TEST_IMG_DIR = os.path.join(DATASET_ROOT, "images/test")
TEST_LABEL_DIR = os.path.join(DATASET_ROOT, "labels/test")

def visualize_comparison(num_samples=5):
    if not os.path.exists(MODEL_PATH):
        print("❌ 학습된 모델 파일이 없습니다.")
        return

    model = YOLO(MODEL_PATH)
    files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png'))]
    random.shuffle(files)

    samples = files[:num_samples]

    plt.figure(figsize=(15, 5 * num_samples))

    for i, file in enumerate(samples):
        img_path = os.path.join(TEST_IMG_DIR, file)
        label_path = os.path.join(TEST_LABEL_DIR, os.path.splitext(file)[0] + ".txt")

        # 이미지 로드
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        # -------------------------------------------------
        # 1. Ground Truth (초록색) 그리기
        # -------------------------------------------------
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    cls, x, y, bw, bh = map(float, line.strip().split())
                    # YOLO xywhn -> pixel xymn (좌상단)
                    x1 = int((x - bw / 2) * w)
                    y1 = int((y - bh / 2) * h)
                    x2 = int((x + bw / 2) * w)
                    y2 = int((y + bh / 2) * h)

                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3) # Green
                    cv2.putText(img, "GT", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # -------------------------------------------------
        # 2. Prediction (빨간색) 그리기
        # -------------------------------------------------
        results = model(img_path, conf=0.1, verbose=False) # Threshold 낮게 설정
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])

                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3) # Red
                cv2.putText(img, f"Pred {conf:.2f}", (x1, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # 출력
        plt.subplot(num_samples, 1, i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Sample: {file} (Green: GT, Red: Prediction)", fontsize=14)

    plt.tight_layout()
    plt.show()


In [ ]:
# 실행
visualize_comparison(1)